# NaiveExactMatching-MatchingRealReads

In [1]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2024-11-18 14:12:14--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.225.141.205, 13.225.141.118, 13.225.141.46, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.225.141.205|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: ‘phix.fa’

phix.fa             100%[===================>]   5.40K  --.-KB/s    in 0s      

2024-11-18 14:12:15 (481 MB/s) - ‘phix.fa’ saved [5528/5528]



In [2]:
import unittest
import collections
class TestMatchingArtificialReads(unittest.TestCase):
    def test_readGenome(self):
        self.assertEqual(len(readGenome('phix.fa')), 5386, 'length of genome string for file phix.fa is wrong! Correct length should be: 5386')
    def test_naive(self):
        self.assertEqual(naive('AG','AGCTTAGATAGC'), [0, 5, 9], 'Tested naive with parameters: p="AG" and t="AGCTTAGATAGC". Should return this list of occurences: [0, 5, 9]')
    def test_generate_reads(self):
        self.test_readGenome()
        genome = readGenome('phix.fa')
        reads = generateReads(genome, 100, 100)
        self.assertEqual(len(reads), 100, 'Generated wrong number of reads! Should be 100 is {}'.format(len(reads)))
        for read in reads:
            self.assertEqual(len(read), 100, 'Read has wrong length. Should be 100 is {}'.format(len(read)))
    def test_readFastq(self):
        phix_reads = readFastq('ERR266411_1.first1000.fastq')
        count = collections.Counter()
        for read in phix_reads:
            count.update(read)
        self.assertEqual(count['T'], 30531, 'Count for T is wrong. Should be 30531 but is {}'.format(count['T']))
        self.assertEqual(count['A'], 28426, 'Count for A is wrong. Should be 30531 but is {}'.format(count['A']))
        self.assertEqual(count['C'], 21890, 'Count for C is wrong. Should be 30531 but is {}'.format(count['C']))
        self.assertEqual(count['G'], 19147, 'Count for G is wrong. Should be 30531 but is {}'.format(count['G']))
        self.assertEqual(count['N'], 6, 'Count for N is wrong. Should be 30531 but is {}'.format(count['N']))

In [4]:
def readGenome(filename):
    """ Reads dna data from a given file
    param: filename name of a fasta file
    returns: dna data without the ids of the fasta file
    rtype: string
    """
    # copy from notebook 1.06
    reads = []
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [5]:
phix_reads = readGenome('phix.fa')

In [6]:
def naive(p, t):
    """Implementation of a 'naive' pattern search in text t
    param: p search-pattern
    param: t text to search in
    returns: list of occurences
    """
    occurrences = []
    for i in range(len(t)-len(p)):
      match = True
      for j in range(len(p)):
        if t[i+j] != p[j]:
          match = False
          break
      if match:
        occurrences.append(i)
    return occurrences

In [8]:
import random
def generateReads(genome, numReads, readLen):
    """Generate reads from random positions in the given genome.
    param: genome dna data from which reads should be generated
    param: numReads defines how many different reads are generated
    param: readLen defines the length of each read
    returns: list containing the generated reads
    """
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome)-readLen) - 1
        reads.append(genome[start : start+readLen])

    return reads

In [9]:
tests = TestMatchingArtificialReads()
suite = unittest.TestSuite()
suite.addTest(TestMatchingArtificialReads('test_readGenome'))
suite.addTest(TestMatchingArtificialReads('test_naive'))
suite.addTest(TestMatchingArtificialReads('test_generate_reads'))
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.011s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [11]:
genome = readGenome("phix.fa")

# Generate 100 reads of length 100
reads = generateReads(genome, 100, 100)

# Count how many reads match the genome exactly
numMatched = 0
for r in reads:
    matches = naive(r, genome)
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, len(reads)))

100 / 100 reads matched the genome exactly!


In [12]:
# On Windows you have to manually download the file
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq

--2024-11-18 14:22:28--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.225.141.205, 13.225.141.183, 13.225.141.118, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.225.141.205|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254384 (248K) [audio/mpeg]
Saving to: ‘ERR266411_1.first1000.fastq’

ERR266411_1.first10 100%[===================>] 248.42K  1.24MB/s    in 0.2s    

2024-11-18 14:22:29 (1.24 MB/s) - ‘ERR266411_1.first1000.fastq’ saved [254384/254384]



In [17]:
def readFastq(filename):
    """ Reads a FastQ File
    param filename: name of the fastq file
    returns: list containing all the dna sequences from the fastq file
    """
    sequences = []
    with open(filename, "r") as fh:
        lines = fh.readlines()
        for line in lines[1:-1:4]:
            seq = line.rstrip()
            sequences.append(seq)
    return sequences

In [18]:
tests = TestMatchingArtificialReads()
suite = unittest.TestSuite()
suite.addTest(TestMatchingArtificialReads('test_readFastq'))
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.013s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [19]:
numMatched = 0
n = 0
for r in phix_reads:
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

5386 / 5386 reads matched the genome exactly!


In [20]:
# Now let's try matching just the first 30 bases of each read
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

5386 / 5386 reads matched the genome exactly!


In [21]:
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [22]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    matches.extend(naive(reverseComplement(r), genome))
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

5386 / 5386 reads matched the genome exactly!


### Compare the different results on the matches!
Think of the different numbers of matches and why these numbers arise. Make some notes!